<a href="https://colab.research.google.com/github/GabeMaldonado/AIforMedicine/blob/master/AIforMed_C2_Theory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notes on AI for Medicine Specialization, Coursera
## Course II -- AI for Medical Prognosis

Prognosis is a branch of medicine that specializes in predicting the future health of patients. For example, given a patient's labs results-- can we estimate the risks of having a heart attack over the next five or ten years. 
Machine Learning is a powerful tool for prognosis and can provide a temendous boost to this branch of medicine by using many different types of medical data to make accurate predictions about a patient's future health. 
Making prognosis is a clinically useful task for a variety of reasons. 
1.   **Risk of Illness** It is useful for informing patients the risk of develping an illness. There are blood tests that are used to estimate the risk of developing breasts and ovarian cancer.
2.   **SUrvival with Illness** It is also used to inform patients how long they can expect to survive with a certain illness. Cancer staging gives an estimate of survival time of a patient with a particular type of cancer. 

Prognosis is also used for guiding treatment. In clinical practice, the prediction of a 10-year risk of heart attack is used to determine whether that patient should get drugs to reduce that risk. Another example is the **6-month mortality risk**. This is used for patients with terminal conditions that have become advanced and uncurable and it is used to determine who should recieve end-of-life care. 

### Prognosis -- Inputs and Outputs

Prognostic models can be thought of as a system that takes in a profile of a patient as an input and outputs s **risk score** for that patient. The patient profile can include:
*   Clinical History which includes major illness, previous medical procedures. 
*   Physical Examinations -- includes vital signs such as temperature and blodd pressure. 
*   Labs and imaging -- inlcudes blood work and CT scans, etc. 
The prognostic model can take one or more of these pieces of information and produce the patient's risk score which can be an arbitrary number or a probability. 

### Calculating $CHA_{2}DS_{2}-VASc$ (Chads vasc) Score

Here we will calculate the risk factor for patients with **atrial fibrilation**. 
Atrial Fibrilation is a common abnormal heart rhythm that puts the patient at the risk of stroke. (Stroke is when the blood flow to a region of the brain is cut off). The CHADS VASC model is used to calculate the one year risk of stroke for patients with AF. The name comes from the following conditions and the numbers in the columns represent the coefficients for each condition and the patient score for each condition (1 = yes, 0 = no). The last column is the coefficient * value and these products get added to obtain the risk factor.

<pre>
*   C - Congestive Heart Failure         1     0     0
*   H - Hypertension                     1     1     1
*   A2 - Age 75 years or older           2     0     0
*   D - Diabetis mellitus                1     1     1
*   S2 - Stroke, TIA or TE               2     0     0
*   V - Vascular disease                 1     0     0
*   A - Age 65 to 74                     1     1     1
*   Sc - Sex category (female)           1     0     0
*                                                   ---
*                                                    3
</pre>

### Calculating the Model for End-stage liver disease (MELD) Score

This score gives an  estimate of the 3-month mortality for patients older than 12 who are on liver transplant waiting lists. This score is a factor in determining how quickly a patient can get a liver transplant. 

Let's calculat the score for a 50 year old woman with the following lab results:
*   Creatinine = 1.0 mg/dL
*   Bilirubin total = 2.0 mg/dL
*   INR = 1.1

<pre>
                     Coeff    Value        Coeff * Value
ln(Creatinine)       0.957    ln(1.0)            0
ln(Bilirubin total)  0.378    ln(2.0)           0.26 
ln(INR)              1.120    ln(1.1)           0.11
Intercept            0.643       1              0.643
-----------------------------------------------------------
Score * 10                                 1.01x10 = 10   
</pre>

This score of 10 is not directly telling us the probability of survival at 3 months but it is informative when comparing it to MELD score of other patients. 






[Check out this notebook to see how to calculate risk scores in python ](https://github.com/GabeMaldonado/AIforMedicine/blob/master/AIforMed_C2_W1_Lab2.ipynb)

[Combining Features in pandas](https://github.com/GabeMaldonado/AIforMedicine/blob/master/AIforMed_C2_W1_Lab3.ipynb)

## Evaluating Prognostic Models

Teh basic idea behind evaluating a prognostinc model is to see how well it performs on pairs of patients. Now, to evaluate these risks scores we need to know whether the patients actually had the event. Here we are looking at death within 10 years so we need to know if patient **A** died within the next 10 years but patient **B** did not. A good prognostic model should give patient A a higher score than to patient B. 
In general when the patient with the worst outcome has a higher risk score, this pair is called ***Concordant***. Now, when the patient with the worst outcome does not have a higher score the pair is called ***Not Concordant/Discordant***. Patient with same scores are called **risk ties**.
A pair where the outcomes are different is called a **permissible** pair. It's with such pairs that we can evaluate prognostic models. 

#### Evaluating the prognostic model:

#### C-Index
*   +1 for a permissible pair that is concordant
*   +0.5 for a permissible pair for risk tie. 

$$ C Index = \frac{\# concordant\  pairs + 0.5 \times \# risk \ ties }{\# permissible\  pairs} $$

### C-Index Interpretation 

$$ P (score (A) > score(B)  | Y_{A}>Y_{B}$$
*   Random Model score = 0.5
*   Perfect model score = 1.0


[Calculating C-Index Lab](https://github.com/GabeMaldonado/AIforMedicine/blob/master/AIforMed_C2_W1_Lab4.ipynb)

## Decision Trees for Prognosis

We can use Decision Trees to build machine learning models. Decision trees are extremely useful in medical applications due to their ability to handle both continuous and categorical data, their interpretability, and the speed at which we can train them. 

Often ML models are considered blac boxes due to their complex inner workings, but in medicine, the ability to explain interpret a model may be critical for human acceptance and trust. 
We will build a prognostic model using age and systolic blood pressure that will predict a 10-year mortality risk. 

*   Systolic blood pressure is the pressure in your blood vessels when your heart beats. 

In this case, the decision tree would divide the input space into three regions of high-risk and low-risk using vertical and horizontal boundaries. The classifier can also be represented as a tree with an if-then structure. The decision tree is asking a series of questions and classifies the patient based on the answers to such questions. 

**How's a decision tree built?**
At a high level:
*   Pick a variable  and a value of that variable that partitions the data, such that one partition contains mostly red and the other partition contains mostly blue. We pick the variable and the value based on how well it splits the data, blues to one side and reds to the other. 
*   Repeat the same process in another region of the data until the partitions are mostly red and blue. 
*   Estimate risk in each partition. 
*   Binarize the output to just output whether an area is low or high risk. We can call a prediction high-risk if the predicted probability is greater than 50% and low-risk otherwise. 

### Challenges when building Decision Trees

One major challenge when bulding decision trees is that if we don't stop growing the decision trees, they continue to create more and more partitions and they can get overly complex. Decision models can create overly complex trees that fit the training data almost prefectly! This ends up being a bad thing adn it is known as ***overfitting***. The model fits the data so well that is unable to generalize other samples or real-world data. 
One way to combat overfitting is to control when we stop growing the trees. We cna control this by setting the maximum depth the tree can grow to. Another popular way to combat overfitting is y building a ***Random Forest***. 

### Random Forests 

Random Forests construct multiple decision trees and average their risk predictions. 
How can a random forest be trained? There are two key concepts when training a random forests. 
1.   Each tree in the forest is constructed using a random sample of the patients. For instance, for the 1st tree, we might draw P1, P2 & P1 which can happen because the random forest samples with replacement.
2.   The random forest algorithm also modifies the splitting procedure in the construction of the decision tree such that it uses a subset of features when creating decision boundaries. 

Random Forests generally boost the performance over single trees. With a single decision tree we might get a test accuracy of 0.71 with a random forest of 100 trees we can get an accuracy of 0.76. 

Random Forests are called an ***ensemble*** learning method because they use multiple decision trees to obtain better prediction performance than could be obtain from any of the decision trees alone. 

There are other popular algorithms that use ensembles including ** Gradient Boosting, XGBoost, LightGBM** which are also able to achieve high performance when working with structure data in medicine and other domains.  